# Chapitre 1

Ce Notebook est une brève introduction aux échecs informatiques modernes. Les moteurs d'échecs utilisant l'intelligence artificielle basée sur l'apprentissage profond (deep learning) ont eu un impact considérable sur la communauté des joueurs d'échecs. AlphaZero, le monstre d'échecs développé par DeepMind, la filiale de recherche de Google, a soudain semblé capable de jouer de belles parties d'échecs, presque semblables à celles d'un humain. Certaines parties montraient des sacrifices de pièces sans gain tactique immédiat, mais des avantages positionnels à long terme que les moteurs d'échecs courants étaient incapables de trouver. Nous commencerons par examiner les méthodes de recherche utilisées par les moteurs d'échecs. Nous apprenons comment les moteurs d'échecs "classiques" utilisent la recherche alpha-bêta, et comment la recherche par arbre de Monte-Carlo offre une alternative pour les échecs et d'autres jeux plus difficiles tels que le Shogi ou le Go. À titre d'exemple, nous mettrons en œuvre la recherche alpha-bêta et la recherche arborescente de Monte-Carlo et nous les testerons avec une position d'échecs simple.

Les échecs se composent de tactique et de stratégie. Pour la tactique, vous calculez les coups et les réponses et vous regardez si l'une des séquences de coups conduit à un avantage ou à un désavantage forcé. Il s'agit essentiellement d'une recherche parmi toutes les variantes possibles. La stratégie consiste à juger une position sans trop de calculs. Ici nous nous concentrons sur la tactique.

## Algorithmes de recherche
### Algorithme minmax

On commence par implémenter l'algorithme minimax. Ce code implémente un joueur d'échecs artificiel basé sur un algorithme de recherche appelé minimax avec élagage alpha-bêta. Après une importation de la bibliothèque d'échecs (chess), on commence par définir une table de valeurs pour évaluer la position des pièces sur l'échiquier. Ici c'est très simplifier : plus la pièce est au centre, meilleur est sa postion et inversement.

In [16]:
import chess

In [17]:
pieceSquareTable = [
  [ -50,-40,-30,-30,-30,-30,-40,-50 ],
  [ -40,-20,  0,  0,  0,  0,-20,-40 ],
  [ -30,  0, 10, 15, 15, 10,  0,-30 ],
  [ -30,  5, 15, 20, 20, 15,  5,-30 ],
  [ -30,  0, 15, 20, 20, 15,  0,-30 ],
  [ -30,  5, 10, 15, 15, 10,  5,-30 ],
  [ -40,-20,  0,  5,  5,  0,-20,-40 ],
  [ -50,-40,-30,-30,-30,-30,-40,-50 ] ]

Ensuite on définit une fonction eval(board) qui évalue la position actuelle de l'échiquier en attribuant des valeurs aux pièces en fonction de leur type et de leur position.

In [18]:
# Définition d'une fonction pour évaluer la position actuelle de l'échiquier
def eval(board):
    # Initialisation des scores pour les blancs et les noirs
    scoreWhite = 0
    scoreBlack = 0
    
    # Parcours de chaque case de l'échiquier
    for i in range(0, 8):
        for j in range(0, 8):
            # Obtention de la case (i, j) sur l'échiquier
            squareIJ = chess.square(i, j)
            # Obtention de la pièce sur la case (i, j)
            pieceIJ = board.piece_at(squareIJ)
            
            # Évaluation de la pièce obtenue
            if str(pieceIJ) == "P":  # Si la pièce est un pion blanc
                scoreWhite += (100 + pieceSquareTable[i][j])  # Ajoute la valeur du pion au score des blancs
            if str(pieceIJ) == "N":  # Si la pièce est un cavalier blanc
                scoreWhite += (310 + pieceSquareTable[i][j])  # Ajoute la valeur du cavalier au score des blancs
            if str(pieceIJ) == "B":  # Si la pièce est un fou blanc
                scoreWhite += (320 + pieceSquareTable[i][j])  # Ajoute la valeur du fou au score des blancs
            if str(pieceIJ) == "R":  # Si la pièce est une tour blanche
                scoreWhite += (500 + pieceSquareTable[i][j])  # Ajoute la valeur de la tour au score des blancs
            if str(pieceIJ) == "Q":  # Si la pièce est une reine blanche
                scoreWhite += (900 + pieceSquareTable[i][j])  # Ajoute la valeur de la reine au score des blancs
            
            if str(pieceIJ) == "p":  # Si la pièce est un pion noir
                scoreBlack += (100 + pieceSquareTable[i][j])  # Ajoute la valeur du pion au score des noirs
            if str(pieceIJ) == "n":  # Si la pièce est un cavalier noir
                scoreBlack += (310 + pieceSquareTable[i][j])  # Ajoute la valeur du cavalier au score des noirs
            if str(pieceIJ) == "b":  # Si la pièce est un fou noir
                scoreBlack += (320 + pieceSquareTable[i][j])  # Ajoute la valeur du fou au score des noirs
            if str(pieceIJ) == "r":  # Si la pièce est une tour noire
                scoreBlack += (500 + pieceSquareTable[i][j])  # Ajoute la valeur de la tour au score des noirs
            if str(pieceIJ) == "q":  # Si la pièce est une reine noire
                scoreBlack += (900 + pieceSquareTable[i][j])  # Ajoute la valeur de la reine au score des noirs
    
    # Retourne la différence entre le score des blancs et celui des noirs
    return scoreWhite - scoreBlack


Un jeu de somme nulle est un jeu où la somme des gains et des pertes de tous les joueurs est égale à 0. Cela signifie donc que le gain de l'un constitue obligatoirement une perte pour l'autre. Par exemple si l'on définit le gain d'une partie d'échecs comme 1 si on gagne, 0 si la partie est nulle et -1 si on perd, le jeu d'échecs est un jeu à somme nulle ([Wikipédia 23/04/2024](https://fr.wikipedia.org/wiki/Jeu_%C3%A0_somme_nulle)).

Comme l'explique la page [wikipédia](https://fr.wikipedia.org/wiki/Algorithme_minimax) (23/04/2024), l'algorithme minimax (aussi appelé algorithme MinMax) est un algorithme qui s'applique à la théorie des jeux pour les jeux à deux joueurs à somme nulle (et à information complète) consistant à minimiser la perte maximum (c'est-à-dire dans le pire des cas). Pour une vaste famille de jeux, le théorème du minimax de von Neumann assure l'existence d'un tel algorithme, même si dans la pratique il n'est souvent guère aisé de le trouver. 

Il amène l'ordinateur à passer en revue toutes les possibilités pour un nombre limité de coups et à leur assigner une valeur qui prend en compte les bénéfices pour le joueur et pour son adversaire. Le meilleur choix est alors celui qui minimise les pertes du joueur tout en supposant que l'adversaire cherche au contraire à les maximiser (le jeu est à somme nulle).

Il existe différents algorithmes basés sur MinMax permettant d'optimiser la recherche du meilleur coup en limitant le nombre de nœuds visités dans l'arbre de jeu, le plus connu est l'élagage alpha-bêta. En pratique, l'arbre est souvent trop vaste pour pouvoir être intégralement exploré (comme pour le jeu d'échecs ou de go). Seule une fraction de l'arbre est alors explorée.
Dans le cas d'arbres très vastes, comme pour les échecs, une IA (système expert, évaluation par apprentissage à partir d'exemple, etc.) peut servir à élaguer certaines branches sur la base d'une estimation de leur utilité. Nous verrons cela dans l'autre chapitre.

Maintenant qu'on a une manière d'évaluer le plateau, on implémente l'algorithme minimax pour évaluer les coups possibles et choisir le meilleur coup à jouer.

In [19]:
# Initialisation d'un compteur de nœuds
NODECOUNT = 0

# Définition de la fonction minimax pour évaluer les coups possibles
def minimax(board, depth, maximize):
    global NODECOUNT  # Utilisation de la variable globale NODECOUNT
    
    # Si c'est un échec et mat
    if(board.is_checkmate()):
        # Si c'est au tour des blancs, retourne une valeur très négative
        if(board.turn == chess.WHITE):
            return -100000
        # Sinon, retourne une valeur très positive
        else:
            return 1000000
    
    # Si c'est un pat ou si le matériel est insuffisant, retourne 0
    if(board.is_stalemate()) or board.is_insufficient_material():
        return 0
    
    # Si la profondeur de recherche est égale à 0, retourne l'évaluation de la position actuelle
    if depth == 0:
        return eval(board)
    
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    
    # Si on cherche à maximiser le score
    if(maximize):
        bestVal = -9999  # Initialisation de la meilleure valeur à une valeur très négative
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à minimax pour évaluer le coup suivant avec une profondeur réduite
            bestVal = max(bestVal, minimax(board, depth - 1, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
        return bestVal  # Retourne la meilleure valeur trouvée
    
    # Si on cherche à minimiser le score
    else:
        bestVal = 9999  # Initialisation de la meilleure valeur à une valeur très positive
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à minimax pour évaluer le coup suivant avec une profondeur réduite
            bestVal = min(bestVal, minimax(board, depth - 1, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
        return bestVal  # Retourne la meilleure valeur trouvée


Définition d'une fonction getNextMove(depth, board, maximize) qui prend en compte la profondeur de recherche et renvoie le meilleur coup possible pour un joueur donné.


In [20]:
# Définition de la fonction pour obtenir le prochain meilleur coup à jouer
def getNextMove(depth, board, maximize):
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    # Initialisation du meilleur coup et de sa valeur associée
    bestMove = None
    bestValue = -9999  # Initialise la meilleure valeur à une valeur très négative
    
    # Si on cherche à minimiser le score, initialise la meilleure valeur à une valeur très positive
    if not maximize:
        bestValue = 9999
        
    # Parcours de tous les coups légaux
    for move in legals:
        board.push(move)  # Joue le coup
        # Évalue le score associé au coup actuel en utilisant l'algorithme minimax
        value = minimax(board, depth - 1, (not maximize))
        board.pop()  # Annule le coup pour revenir à l'état précédent
        
        # Met à jour le meilleur coup et sa valeur associée en fonction de la stratégie (maximisation ou minimisation)
        if maximize:  # Si on cherche à maximiser le score
            if value > bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
        else:  # Si on cherche à minimiser le score
            if value < bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
    
    # Retourne le meilleur coup trouvé et sa valeur associée
    return (bestMove, bestValue)


Maintenant on peut exécuter des commandes qui simulent un jeu d'échecs avec quelques mouvements (coup du berger) et affichage du meilleur coup calculé par l'algorithme.


In [21]:
board = chess.Board()
board.push_san("e4")
board.push_san("e5")
board.push_san("Qh5")
board.push_san("Nc6")
board.push_san("Bc4")
board.push_san("Nf6")
print(board)

r . b q k b . r
p p p p . p p p
. . n . . n . .
. . . . p . . Q
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B . K . N R


On exécute notre code pour voir le résultat.

In [22]:
print(getNextMove(4, board, True))
print(NODECOUNT)

(Move.from_uci('h5f7'), 1000000)
1337211


On obtient bien le meilleur coup : dame en f7 délivrant un échec et mat en 1 coup à l'aide du coup du berger. Cependant il y a 1337211 positions évaluées en environ 3min pour trouver ce coup. C'est beaucoup trop pour un jeu d'échecs complet. On va donc implémenter un algorithme plus rapide pour évaluer les coups possibles.

### Algorithme Alpha-Bêta

On peux cependant optimiser cette algorithme sans modifier le résultat en utilisant l'élagage alpha-bêta. L’élagage alpha-bêta (abrégé élagage αβ) est une technique permettant de réduire le nombre de nœuds évalués par l'algorithme minimax. Lors de l'exploration, il n'est pas nécessaire d'examiner les sous-arbres qui conduisent à des configurations dont la valeur ne contribuera pas au calcul du gain à la racine de l'arbre. Dit autrement, l'élagage αβ n'évalue pas des nœuds dont on peut penser, si la fonction d'évaluation est à peu près correcte, que leur qualité sera inférieure à celle d'un nœud déjà évalué. On peut facilement le voir avec l'image suivante :
![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/Poda_alfa-beta.svg/2560px-Poda_alfa-beta.svg.png)

On réutilise notre pieceSquareTable et notre fonction eval. On code maintenant Alpha-Bêta :

In [26]:
# Définition d'un compteur de nœuds
NODECOUNT = 0

# Définition de la fonction alpha-beta pour évaluer les coups possibles
def alphaBeta(board, depth, alpha, beta, maximize):
    global NODECOUNT  # Utilisation de la variable globale NODECOUNT
    
    # Si c'est un échec et mat
    if(board.is_checkmate()):
        # Si c'est au tour des blancs, retourne une valeur très négative
        if(board.turn == chess.WHITE):
            return -100000
        # Sinon, retourne une valeur très positive
        else:
            return 1000000
    
    # Si la profondeur de recherche est égale à 0, retourne l'évaluation de la position actuelle
    if depth == 0:
        return eval(board)
    
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    
    # Si on cherche à maximiser le score
    if(maximize):
        bestVal = -9999  # Initialisation de la meilleure valeur à une valeur très négative
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à alphaBeta pour évaluer le coup suivant avec une profondeur réduite
            bestVal = max(bestVal, alphaBeta(board, depth - 1, alpha, beta, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
            alpha = max(alpha, bestVal)  # Met à jour la valeur alpha
            if alpha >= beta:  # Si alpha est supérieur ou égal à beta
                return bestVal  # Effectue une coupure alpha et retourne la meilleure valeur trouvée
        return bestVal  # Retourne la meilleure valeur trouvée
    
    # Si on cherche à minimiser le score
    else:
        bestVal = 9999  # Initialisation de la meilleure valeur à une valeur très positive
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à alphaBeta pour évaluer le coup suivant avec une profondeur réduite
            bestVal = min(bestVal, alphaBeta(board, depth - 1, alpha, beta, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
            beta = min(beta, bestVal)  # Met à jour la valeur beta
            if beta <= alpha:  # Si beta est inférieur ou égal à alpha
                return bestVal  # Effectue une coupure beta et retourne la meilleure valeur trouvée
        return bestVal  # Retourne la meilleure valeur trouvée

On implémente maintenant notre nouvelle fonction getNextMoveAlphaBeta :

In [24]:
# Définition de la fonction pour obtenir le prochain meilleur coup à jouer
def getNextMoveAlphaBeta(depth, board, maximize):
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    
    # Initialisation du meilleur coup et de sa valeur associée
    bestMove = None
    bestValue = -9999  # Initialise la meilleure valeur à une valeur très négative
    
    # Si on cherche à minimiser le score, initialise la meilleure valeur à une valeur très positive
    if not maximize:
        bestValue = 9999
    
    # Parcours de tous les coups légaux
    for move in legals:
        board.push(move)  # Joue le coup
        # Évalue le score associé au coup actuel en utilisant l'algorithme alpha-beta
        value = alphaBeta(board, depth - 1, -10000, 10000, (not maximize))
        board.pop()  # Annule le coup pour revenir à l'état précédent
        
        # Met à jour le meilleur coup et sa valeur associée en fonction de la stratégie (maximisation ou minimisation)
        if maximize:  # Si on cherche à maximiser le score
            if value > bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
        else:  # Si on cherche à minimiser le score
            if value < bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
    
    # Retourne le meilleur coup trouvé et sa valeur associée
    return (bestMove, bestValue)

On peut maintenant faire un test avec la position précédente :

In [27]:
print(getNextMoveAlphaBeta(4, board, True))
print(NODECOUNT)

(Move.from_uci('h5f7'), 1000000)
119034


On obtient bien le même résultat, l'échec et mat avec Dame en f7, cepandant on a évalué 119034 positions en environ 15 secondes au lieu de 1337211 et 3 minutes. C'est déjà beaucoup mieux mais on peut faire encore mieux avec un autre algorithme de recherche.


Il est difficile de déterminer un niveau ELO précis pour l'algorithme minimax aux échecs car cela dépend de plusieurs facteurs : 
- Profondeur de recherche
- Fonction d'évaluation
- Optimisations



En résumé, la force de l'algorithme minimax aux échecs varie considérablement en fonction de sa mise en œuvre et des optimisations employées, se situant généralement entre 1500 et 2200 ELO pour les versions basiques et pouvant dépasser les 2200 ELO pour les implémentations plus avancées.

### Algorithmes de recherche Monte Carlo

En algorithmique, une heuristique est une méthode de calcul qui fournit rapidement une solution réalisable, pas nécessairement optimale ou exacte, pour un problème d'optimisation difficile.
En informatique, et plus précisément en intelligence artificielle, la recherche arborescente Monte Carlo ou Monte Carlo tree search (MCTS) est un algorithme de recherche heuristique utilisé dans le cadre de la prise de décision ([Wikipédia 23/04/2024](https://fr.wikipedia.org/wiki/Monte_Carlo_tree_search)).

L'algorithme MCTS est un algorithme qui explore l'arbre des possibles. La racine est la configuration initiale du jeu. Chaque nœud est une configuration et ses enfants sont les configurations suivantes. MCTS conserve en mémoire un arbre qui correspond aux nœuds déjà explorés de l'arbre des possibles. Une feuille de cet arbre (un nœud n'ayant pas d'enfants) est soit une configuration finale (i.e. on sait si un des joueurs a gagné, ou s'il y a match nul), soit un nœud dont aucun enfant n'a encore été exploré. Dans chaque nœud, on stocke deux nombres : le nombre de simulations gagnantes, et le nombre total de simulations. Chaque étape est composée de quatre phases.

- Sélection. Depuis la racine, on sélectionne successivement des enfants jusqu'à atteindre une feuille. Dans cette phase, le choix des enfants est guidé par un compromis entre exploitation (aller vers un enfant qui a été prouvé comme prometteur) et exploration (aller visiter un autre enfant, qui a l'air moins prometteur mais qui pourrait l'être davantage). Dans l'exemple donné dans la figure, on choisit la feuille de gauche (de profondeur 3).
- Expansion: si cette feuille n'est pas finale, créer un enfant (ou plusieurs) en utilisant les règles du jeu et choisir l'un des enfants. Sur l'exemple, on ajoute cet enfant, marqué 0/0.
- Simulation: simuler une exécution d'une partie au hasard depuis cet enfant, jusqu'à atteindre une configuration finale.
- Rétropropagation (Backpropagation): utiliser le résultat de la partie au hasard et mettre à jour les informations sur la branche en partant du nœud enfant vers la racine. Sur l'exemple, la partie était perdante pour blanc. On incrémente donc uniquement le nombre de simulations totales sur la branche pour les nœuds blancs et on incrémente le nombre de simulations gagnées et le nombre de simulations totales pour les noirs sur la branche : 0/0 devient 0/1, 3/3 devient 3/4, 5/6 devient 5/7, 7/10 devient 7/11, et 12/21 devient 12/22. Si la partie avait été gagnante : 0/0 serait devenu 1/1, 3/3 serait devenu 4/4, 5/6 serait devenu 6/7, 7/10 serait devenu 8/11, et 12/21 serait devenu 13/22.

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/MCTS_%28French%29.svg/2560px-MCTS_%28French%29.svg.png)

On commence par importer les bibliothèques nécessaires et initialiser les variables.

In [8]:
import random
import chess
import math

random.seed(42)

PLAYER = chess.WHITE
OPPONENT = chess.BLACK

On crée une class python TreeNode. Cette classe représente un nœud dans un arbre de recherche utilisé dans l'algorithme Monte Carlo Tree Search (MCTS). Chaque nœud contient des informations sur le nombre de visites (M), la valeur (V), les mouvements déjà visités et les nœuds enfants associés (visitedMovesAndNodes), les mouvements légaux non encore visités (nonVisitedLegalMoves), l'échiquier associé (board) et le nœud parent (parent). La classe comprend également deux méthodes : isMCTSLeafNode, qui vérifie si le nœud est une feuille dans l'algorithme MCTS, et isTerminalNode, qui vérifie si le nœud est un nœud terminal. 

In [9]:
# Définition de la classe TreeNode
class TreeNode():
    
    # Méthode d'initialisation de la classe
    def __init__(self, board):
        self.M = 0  # Nombre de visites de ce nœud dans le cadre de l'algorithme MCTS (Monte Carlo Tree Search)
        self.V = 0  # Valeur de ce nœud dans le cadre de l'algorithme MCTS
        self.visitedMovesAndNodes = []  # Liste des mouvements visités et des nœuds enfants associés
        self.nonVisitedLegalMoves = []  # Liste des mouvements légaux non encore visités
        self.board = board  # L'échiquier associé à ce nœud
        self.parent = None  # Nœud parent de ce nœud
        
        # Initialisation de la liste des mouvements légaux non encore visités
        for m in self.board.legal_moves:
            self.nonVisitedLegalMoves.append(m)
    
    # Méthode pour vérifier si le nœud est une feuille de l'algorithme MCTS
    def isMCTSLeafNode(self):
        return len(self.nonVisitedLegalMoves) != 0
    
    # Méthode pour vérifier si le nœud est un nœud terminal
    def isTerminalNode(self):
        return len(self.nonVisitedLegalMoves) == 0 and len(self.visitedMovesAndNodes) == 0

Puis on crée une fonction uctValue qui calcule la valeur UCT (Upper Confidence Bound applied to Trees) d'un nœud dans un arbre de recherche utilisé dans l'algorithme Monte Carlo Tree Search (MCTS). La valeur UCT est utilisée pour sélectionner le prochain nœud à explorer dans l'arbre de recherche. La formule utilisée prend en compte le nombre de visites du nœud (node.M), sa valeur (node.V), le nombre de visites de son nœud parent (parent.V) et un coefficient d'exploration.

In [10]:
# Définition de la fonction pour calculer la valeur UCT (Upper Confidence Bound applied to Trees)
def uctValue(node, parent):
    # Calcul de la valeur UCT en utilisant la formule UCT = (M / V) + c * sqrt(ln(P) / V), où :
    # - M est le nombre de visites du nœud
    # - V est la valeur du nœud
    # - P est le nombre de visites du nœud parent
    # - c est un coefficient d'exploration (dans ce cas, 1.4142, approximativement √2)
    val = (node.M / node.V) + 1.4142 * math.sqrt(math.log(parent.V) / node.V)
    # Retourne la valeur UCT calculée
    return val

Ensuite on écrit une fonction select qui sélectionne récursivement le prochain nœud à explorer dans l'arbre de recherche MCTS en utilisant la méthode de la valeur UCT (Upper Confidence Bound applied to Trees). Elle parcourt les nœuds enfants déjà visités et calcule la valeur UCT pour chacun d'eux, puis sélectionne le nœud enfant avec la meilleure valeur UCT. Si aucun enfant n'est trouvé avec une valeur UCT valide, une erreur est déclenchée.

In [11]:
# Définition de la fonction pour sélectionner le prochain nœud à explorer dans l'arbre de recherche
def select(node):
    # Si le nœud est une feuille de l'algorithme MCTS ou un nœud terminal, retourne le nœud
    if node.isMCTSLeafNode() or node.isTerminalNode():
        return node
    else:
        # Initialisation du meilleur enfant selon la valeur UCT et de sa valeur associée
        maxUctChild = None
        maxUctValue = -1000000.  # Initialise la meilleure valeur UCT à une valeur très négative
        
        # Parcours de tous les mouvements visités et les nœuds enfants associés
        for move, child in node.visitedMovesAndNodes:
            # Calcul de la valeur UCT pour l'enfant actuel
            uctValChild = uctValue(child, node)
            # Si la valeur UCT de cet enfant est meilleure que la meilleure valeur UCT trouvée jusqu'à présent
            if uctValChild > maxUctValue:
                maxUctChild = child  # Met à jour le meilleur enfant
                maxUctValue = uctValChild  # Met à jour la meilleure valeur UCT
        
        # Si aucun meilleur enfant n'a été identifié, déclenche une erreur
        if maxUctChild is None:
            raise ValueError("could not identify child with best uct value")
        else:
            # Sélectionne récursivement le meilleur enfant
            return select(maxUctChild)

Maintenant on crée une fonction expand qui étend un nœud de l'arbre de recherche MCTS en créant un nouvel enfant. Elle sélectionne le prochain mouvement légal non encore visité à partir du nœud, joue ce mouvement sur une copie de l'échiquier du nœud parent, crée un nouveau nœud enfant avec cet échiquier, lie ce nouveau nœud au nœud parent et ajoute le mouvement et le nœud enfant associé à la liste des mouvements visités et des nœuds enfants du nœud parent. Enfin, elle retourne le nouveau nœud enfant créé.

In [12]:
# Définition de la fonction pour étendre un nœud en créant un nouvel enfant
def expand(node):
    # Sélectionne le prochain mouvement légal non encore visité à partir du nœud
    moveToExpand = node.nonVisitedLegalMoves.pop()
    
    # Copie l'échiquier du nœud parent pour créer un nouvel échiquier
    board = node.board.copy()
    
    # Joue le mouvement sélectionné sur le nouvel échiquier
    board.push(moveToExpand)
    
    # Crée un nouveau nœud enfant avec le nouvel échiquier
    childNode = TreeNode(board)
    
    # Lie le nouveau nœud enfant au nœud parent
    childNode.parent = node
    
    # Ajoute le mouvement et le nœud enfant associé à la liste des mouvements visités et des nœuds enfants
    node.visitedMovesAndNodes.append((moveToExpand, childNode))
    
    # Retourne le nouveau nœud enfant
    return childNode


Après, on crée une fonction qui simule une partie d'échecs à partir d'un état de jeu donné par un nœud de l'arbre de recherche MCTS. Elle joue des coups aléatoires jusqu'à ce qu'un résultat soit atteint (victoire, match nul ou échec et mat), puis elle détermine le "paiement" associé à ce résultat. Le "paiement" est utilisé pour mettre à jour les statistiques des nœuds lors de la rétropropagation dans l'algorithme MCTS.

Dans le contexte de l'algorithme Monte Carlo Tree Search (MCTS) appliqué aux échecs, le "paiement" (ou "payoff" en anglais) représente une mesure de la qualité d'un nœud dans l'arbre de recherche. Plus précisément, le paiement est une valeur numérique qui est utilisée pour évaluer le résultat d'une simulation de partie effectuée à partir d'un nœud donné.

Dans le cas des échecs, le paiement peut prendre différentes valeurs pour représenter différents résultats de la simulation de partie :

1. **Victoire (ou gain maximal) :** Si la simulation aboutit à une victoire pour le joueur pour lequel on effectue la simulation, le paiement est généralement fixé à 1. Cela indique que la position associée au nœud est très favorable.

2. **Défaite (ou perte maximale) :** Si la simulation aboutit à une défaite pour le joueur pour lequel on effectue la simulation, le paiement est généralement fixé à 0. Cela indique que la position associée au nœud est très défavorable.

3. **Match nul (ou équilibre) :** Si la simulation aboutit à un match nul, le paiement est généralement fixé à 0.5. Cela indique que la position associée au nœud est équilibrée, aucun des joueurs n'ayant d'avantage significatif.

Le paiement est utilisé dans l'algorithme MCTS pour mettre à jour les statistiques des nœuds pendant la phase de rétropropagation, où les résultats de la simulation de partie sont propagés vers le haut de l'arbre pour mettre à jour les estimations de la qualité des mouvements et des positions. Les statistiques des nœuds, telles que le nombre de visites et la valeur associée, sont utilisées pour guider la recherche vers les mouvements les plus prometteurs.

In [13]:
# Définition de la fonction pour simuler une partie à partir d'un nœud donné
def simulate(node):
    # Copie l'échiquier du nœud pour simuler la partie
    board = node.board.copy()
    
    # Boucle de simulation de la partie jusqu'à ce qu'un résultat soit atteint (victoire, match nul ou échec et mat)
    while board.outcome(claim_draw=True) == None:
        # Obtient la liste des mouvements légaux possibles
        legal_moves = list(board.legal_moves)
        # Sélectionne un mouvement aléatoire parmi les mouvements légaux
        move = random.choice(legal_moves)
        # Joue le mouvement sur l'échiquier
        board.push(move)
    
    # Initialise le paiement par défaut à 0.5 (match nul)
    payout = 0.5
    
    # Obtient le résultat de la partie
    outcome = board.outcome(claim_draw=True)
    
    # Met à jour le paiement en fonction du résultat de la partie
    if outcome.winner == PLAYER:  # Si le joueur a gagné
        payout = 1  # Le paiement est de 1 (victoire)
    elif outcome.winner == OPPONENT:  # Si l'adversaire a gagné
        payout = 0  # Le paiement est de 0 (défaite)
    else:  # Si la partie est un match nul
        payout = 0.5  # Le paiement est de 0.5 (match nul)
    
    # Retourne le paiement obtenu à partir de la simulation de la partie
    return payout


Enfin on écrit une fonction qui rétropropage le paiement (ou le "payoff") obtenu à partir d'une simulation de partie effectuée à partir d'un nœud donné à travers les nœuds parents de cet arbre de recherche MCTS. Elle met à jour les statistiques des nœuds parents en ajoutant le paiement au nombre total de victoires (M) du nœud et en incrémentant le nombre total de visites (V) du nœud de 1.

La fonction continue de rétropropager récursivement le paiement vers le nœud parent du nœud actuel tant que le nœud actuel n'est pas la racine de l'arbre (c'est-à-dire tant qu'il a un nœud parent). Cela permet de mettre à jour les statistiques de tous les nœuds traversés lors de la simulation de partie, ce qui aide à estimer la qualité des différents mouvements et positions dans l'arbre de recherche.

In [14]:
# Définition de la fonction pour rétropropager le paiement à travers les nœuds parents de l'arbre de recherche
def backpropagate(node, payout):
    # Mise à jour du nombre total de victoires (M) et du nombre total de visites (V) du nœud actuel
    node.M = node.M + payout  # Ajoute le paiement au nombre total de victoires du nœud
    node.V = node.V + 1  # Incrémente le nombre total de visites du nœud de 1
    
    # Si le nœud actuel n'est pas la racine de l'arbre (i.e., il a un nœud parent)
    if node.parent is not None:
        # Rétropropage récursivement le paiement au nœud parent
        return backpropagate(node.parent, payout)


On peut maitenant tester sur une partie :

In [17]:
board = chess.Board()
board.push_uci("e2e4")
board.push_uci("e7e6")
board.push_uci("d2d4")
board.push_uci("d7d5")
board.push_uci("b1c3")
board.push_uci("g8f6")
board.push_uci("e4e5")
board.push_uci("f6d7")
board.push_uci("g1f3")
board.push_uci("f8b4")
board.push_uci("f1d3")
board.push_uci("e8g8")
print(board)

r n b q . r k .
p p p n . p p p
. . . . p . . .
. . . p P . . .
. b . P . . . .
. . N B . N . .
P P P . . P P P
R . B Q K . . R


On utilise la fonction MCTS pour simuler une partie d'échecs à partir de la position initiale de l'échiquier. On affiche ensuite le nombre de simulations effectuées et le meilleur coup trouvé par l'algorithme MCTS.

In [18]:
# Crée un nœud racine avec l'échiquier initial
root = TreeNode(board)

# Affiche le nombre de mouvements non visités à partir du nœud racine
print("Le nombre de nœuds non visités est : ", len(root.nonVisitedLegalMoves))

Le nombre de nœuds non visités est :  34


Cela veut dire qu'il y a seulement 34 mouvements que l'algorithme n'a pas encore exploré et évalué. On itère sur ces mouvements pour trouver le meilleur coup à jouer.

In [19]:
# Boucle principale pour effectuer des itérations de l'algorithme MCTS
for i in range(0, 5000):
    # Affiche l'avancement toutes les 10 itérations
    if i % 10 == 0:
        print(i)
    
    # Affiche les statistiques de tous les nœuds enfants toutes les 100 itérations
    if i % 100 == 0:
        for move, child in root.visitedMovesAndNodes:
            print("move: " + str(move) + " " + str(child.M) + ", " + str(child.V))
    
    # Sélectionne un nœud à explorer à partir du nœud racine
    node = select(root)
    
    # Si le nœud sélectionné n'est pas un nœud terminal, l'étend en créant un nouvel enfant
    if not node.isTerminalNode():
        node = expand(node)
    
    # Simule une partie à partir du nœud étendu et obtient le paiement associé
    payout = simulate(node)
    
    # Rétropropage le paiement à travers les nœuds parents de l'arbre de recherche
    backpropagate(node, payout)




0
10
20
30
40
50
60
70
80
90
100
move: a2a4 1.5, 3
move: g2g4 3.0, 4
move: h2h4 1.5, 3
move: a2a3 1.5, 3
move: b2b3 1.5, 3
move: g2g3 1.5, 3
move: h2h3 2.0, 4
move: e1g1 1.5, 3
move: a1b1 1.5, 3
move: c1d2 1.5, 3
move: c1e3 1.5, 3
move: c1f4 0.5, 2
move: c1g5 1.5, 3
move: c1h6 1.5, 3
move: d1d2 1.5, 3
move: d1e2 1.5, 3
move: e1f1 1.5, 3
move: e1d2 1.5, 3
move: e1e2 1.0, 3
move: h1f1 1.5, 3
move: h1g1 1.5, 3
move: d3f1 0.5, 2
move: d3e2 0.5, 2
move: d3c4 2.5, 4
move: d3e4 2.0, 3
move: d3b5 1.5, 3
move: d3f5 1.5, 3
move: d3a6 0.5, 2
move: d3g6 1.5, 3
move: d3h7 1.5, 3
move: f3g1 1.5, 3
move: f3d2 1.5, 3
move: f3h4 1.5, 3
move: f3g5 0.5, 2
110
120
130
140
150
160
170
180
190
200
move: a2a4 3.0, 6
move: g2g4 4.5, 7
move: h2h4 4.0, 7
move: a2a3 3.0, 6
move: b2b3 2.0, 5
move: g2g3 4.0, 7
move: h2h3 3.0, 6
move: e1g1 3.0, 6
move: a1b1 4.5, 7
move: c1d2 3.5, 6
move: c1e3 3.0, 6
move: c1f4 1.0, 4
move: c1g5 3.0, 6
move: c1h6 3.0, 6
move: d1d2 2.0, 5
move: d1e2 3.0, 6
move: e1f1 2.5, 6
move: e1d

On affiche les statistiques du nœud racine après les itérations de l'algorithme MCTS :

In [20]:
print(root.M)
print(root.V)

2519.5
5000


Cela signifie que le nœud racine a été visité 5000 fois au total, et que parmi ces visites, il a obtenu un total de 2519.5 "victoires" ou "paiements". On affiche les statistiques de tous les nœuds enfants du nœud racine :

In [21]:
for move, child in root.visitedMovesAndNodes:
    print("move: " + str(move) + " " + str(child.M) + ", " + str(child.V))

move: a2a4 87.0, 166
move: g2g4 83.5, 161
move: h2h4 93.5, 176
move: a2a3 64.0, 132
move: b2b3 77.5, 152
move: g2g3 94.5, 177
move: h2h3 75.5, 149
move: e1g1 72.0, 144
move: a1b1 79.5, 155
move: c1d2 86.5, 166
move: c1e3 55.5, 119
move: c1f4 65.0, 134
move: c1g5 85.0, 163
move: c1h6 85.0, 163
move: d1d2 74.0, 147
move: d1e2 55.0, 119
move: e1f1 64.0, 132
move: e1d2 85.5, 164
move: e1e2 81.0, 158
move: h1f1 75.5, 149
move: h1g1 64.0, 132
move: d3f1 58.0, 123
move: d3e2 66.0, 135
move: d3c4 62.5, 130
move: d3e4 72.0, 144
move: d3b5 68.0, 138
move: d3f5 85.0, 163
move: d3a6 68.0, 138
move: d3g6 81.5, 158
move: d3h7 74.0, 147
move: f3g1 72.0, 144
move: f3d2 70.5, 142
move: f3h4 83.5, 161
move: f3g5 55.5, 119


Ces lignes affichent les statistiques des nœuds enfants du nœud racine, sous la forme "move: [mouvement] [M], [V]". Voici comment interpréter chaque ligne :

"move: [mouvement]": Le mouvement associé au nœud enfant.
"[M]": La somme des paiements obtenus à partir de toutes les visites de ce nœud enfant. Cela représente une estimation de la qualité du mouvement. Une valeur plus élevée indique que ce mouvement a tendance à être associé à de bons résultats lors des simulations.
"[V]": Le nombre total de visites de ce nœud enfant. Cela représente le nombre de fois que ce mouvement a été exploré et évalué.
Par exemple :

"move: a2a4 87.0, 166" indique que le mouvement a2a4 a été associé à une somme de paiements de 87.0 sur un total de 166 visites.
"move: g2g4 83.5, 161" indique que le mouvement g2g4 a été associé à une somme de paiements de 83.5 sur un total de 161 visites.
Ces statistiques sont utilisées par l'algorithme MCTS pour guider la sélection des mouvements les plus prometteurs à partir du nœud racine. Les mouvements avec des valeurs plus élevées de M/V sont considérés comme plus prometteurs et sont plus susceptibles d'être explorés davantage lors des itérations suivantes de l'algorithme MCTS.

On calcule le meilleur coup à jouer en fonction des statistiques des nœuds enfants du nœud racine :

In [33]:
# On parcours la liste des mouvements visités et des nœuds enfants associés pour trouver le meilleur mouvement
#On initialise le meilleur mouvement et sa valeur associée
bestMove = None
bestValue = -9999
for move, child in root.visitedMovesAndNodes:
    # On calcule M/V
    MV = child.M / child.V
    # On met à jour le meilleur mouvement et sa valeur associée
    if MV > bestValue:
        bestValue = MV
        bestMove = move
# On affiche le meilleur mouvement trouvé
print("Le meilleur mouvement est : ", bestMove)
print("La valeur associée est : ", bestValue)



Le meilleur mouvement est :  g2g3
La valeur associée est :  0.5338983050847458


### Conclusion
Alpha-Beta est un puissant algorithme de recherche pour les arbres de jeu. Son inconvénient est qu'il nécessite des connaissances spécifiques au domaine pour la fonction d'évaluation (ce qui est impossible pour le jeu de go). En outre, le facteur de ramification du jeu ne doit pas être trop important - dans le cas extrême, alpha-beta ne recherchera qu'un seul niveau de profondeur et se résumera essentiellement à appliquer directement la fonction d'évaluation une seule fois. La recherche arborescente de Monte-Carlo est indépendante du domaine et peut être appliquée à n'importe quel jeu déterministe à deux joueurs. Si l'alpha-bêta et les MCTS fonctionnent tous deux, l'alpha-bêta est généralement plus précis et plus rapide. Mais pour les jeux dont le facteur de ramification est très élevé et/ou pour lesquels il n'existe pas de bonne fonction d'évaluation, les MCTS peuvent constituer une alternative puissante à la recherche alpha-bêta.